<a href="https://colab.research.google.com/github/patrickhamzaokello/ColabFavourites/blob/main/ProcessAwsUploadedFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3 pydub ffmpeg-python pyloudnorm mutagen
!pip install soundfile numpy
!sudo apt-get install -y ffmpeg
!pip install mysql-connector-python pandas openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 54.7 MB/s eta 0:00:00


In [2]:
import os
import boto3
from pydub import AudioSegment
import ffmpeg
import pyloudnorm as pyln
import numpy as np
from mutagen.easyid3 import EasyID3
from mutagen.oggvorbis import OggVorbis
import logging
from datetime import datetime
import csv
import re
import psycopg2
from psycopg2.extras import RealDictCursor
import mysql.connector
from contextlib import contextmanager

import subprocess
import json
import shutil
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)

logger = logging.getLogger(__name__)

In [3]:
class DatabaseManager:
    """Handle database operations for tracking upload status"""

    def __init__(self, db_config, db_type='mysql'):
        """
        Initialize database connection
        db_config: dict with connection parameters
        db_type: 'mysql' or 'postgresql'
        """
        self.db_config = db_config
        self.db_type = db_type.lower()

    @contextmanager
    def get_connection(self):
        """Get database connection with automatic cleanup"""
        conn = None
        try:
            if self.db_type == 'mysql':
                conn = mysql.connector.connect(**self.db_config)
            elif self.db_type == 'postgresql':
                conn = psycopg2.connect(**self.db_config)
            else:
                raise ValueError(f"Unsupported database type: {self.db_type}")

            yield conn
            conn.commit()
        except Exception as e:
            if conn:
                conn.rollback()
            raise e
        finally:
            if conn:
                conn.close()



    def update_upload_status(self, track_id, status, error_message=None):
        """
        Update the Uploads table with processing status by matching track filename

        Args:
            track_id: Track filename (without extension) to match against database
            status: '0 - unprocessed', '1- processed', '2-failed'
            error_message: error description for failed tracks
        """
        try:
            with self.get_connection() as conn:
                cursor = conn.cursor()

                if self.db_type == 'mysql':
                    cursor = conn.cursor(dictionary=True)
                else:  # postgresql
                    cursor = conn.cursor(cursor_factory=RealDictCursor)

                current_time = datetime.now()


                # First, find the upload_id by matching the track filename
                # Assuming your database has file_name or similar field that contains the filename
                find_query = """
                SELECT upload_id FROM Uploads
                WHERE file_name LIKE %s
                """
                search_patterns = [
                    f"%{track_id}%" # file_name contains the track_id
                ]

                cursor.execute(find_query, search_patterns)
                result = cursor.fetchone()

                if not result:
                    logger.warning(f"No record found for track_id: {track_id}")
                    self._debug_available_tracks(cursor, track_id)
                    return False

                # Get the actual upload_id from the result
                if self.db_type == 'mysql':
                    upload_id = result['upload_id']
                else:
                    upload_id = result[0]

                logger.info(f"Found upload_id {upload_id} for track_id {track_id}")

                # Update the Uploads table based on status
                if status == 'processing':
                    query = """
                    UPDATE Uploads
                    SET processing_status = %s,
                        processing_started_at = %s,
                        error_message = %s
                    WHERE upload_id = %s
                    """
                    params = (4, current_time,error_message, upload_id)

                elif status == 'completed':

                    query = """
                    UPDATE Uploads
                    SET processing_status = %s,
                    error_message = %s,
                        processing_completed_at = %s
                    WHERE upload_id = %s
                    """
                    params = (1,error_message, current_time, upload_id)

                elif status == 'failed':
                    query = """
                    UPDATE Uploads
                    SET processing_status = %s,
                    processing_completed_at = %s,
                        error_message = %s
                    WHERE upload_id = %s
                    """
                    params = (2, current_time,error_message, upload_id)

                cursor.execute(query, params)

                if cursor.rowcount == 0:
                    logger.warning(f"Failed to update record for upload_id: {upload_id}")
                    return False

                logger.info(f"Updated upload {upload_id} status to '{status}' for track {track_id}")
                return True

        except Exception as e:
            logger.error(f"Database update failed for track {track_id}: {str(e)}")
            return False

        except Exception as e:
            logger.error(f"Failed to get upload info for track {track_id}: {str(e)}")
            return None

In [4]:
from google.colab import userdata
db_config = {
      'host': userdata.get('mwonyaDBHost'),
      'user': userdata.get('mwonyaDBUser'),
      'password': userdata.get('mwonyaDBPassword'),
      'database': userdata.get('mwonyaDB')
  }

# Configuration
S3_BUCKET = userdata.get('mwonyaS3_BUCKET')
AWS_ACCESS_KEY = userdata.get('mwonyaAWS_ACCESS_KEY')
AWS_SECRET_KEY = userdata.get('mwonyaAWS_SECRET_KEY')
AWS_REGION = userdata.get('mwonyaAWS_REGION')

# Initialize S3 client
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [5]:
class AudioProcessor:
    def __init__(self, s3_bucket, db_config, db_type='mysql',
                 raw_prefix='tracks/', processed_prefix='mwonya_audio_tracks/',
                 failed_prefix='failedprocessing_tracks/', segment_duration=10):

        # S3 setup
        self.s3 = boto3.client('s3',
                      aws_access_key_id=AWS_ACCESS_KEY,
                      aws_secret_access_key=AWS_SECRET_KEY,
                      region_name=AWS_REGION)
        self.bucket = s3_bucket
        self.raw_prefix = raw_prefix
        self.processed_prefix = processed_prefix
        self.failed_prefix = failed_prefix
        self.segment_duration = segment_duration

        # Database setup
        self.db_manager = DatabaseManager(db_config, db_type)

        # Updated quality profile for AAC
        self.quality_profile = {
            'bitrate': '128k',      # Optimal balance for AAC
            'sample_rate': 44100,   # Standard CD quality
            'codec': 'aac',         # Use AAC instead of Ogg
            'extension': '.m4a'     # Standard extension for AAC files
        }

        # HLS quality configurations (from the first code)
        self.hls_quality_configs = {

            'low': {
                'bitrate': '48k',
                'sample_rate': '24000',
                'channels': '2',
                'description': 'Default quality for music in low-bandwidth environments',
                'codec': 'aac',
                'codec_options': ['-profile:a', 'aac_low']
            },
            'med': {
                'bitrate': '64k',
                'sample_rate': '32000',
                'channels': '2',
                'description': 'Improved quality for users on stronger connections',
                'codec': 'aac',
                'codec_options': ['-profile:a', 'aac_low']
            },
            'high': {
                'bitrate': '128k',
                'sample_rate': '44100',
                'channels': '2',
                'description': 'High quality audio for good network conditions',
                'codec': 'aac',
                'codec_options': ['-profile:a', 'aac_low']
            }
        }

        self.target_loudness = -14  # LUFS for loudness normalization

        logger.info(f"AudioProcessor initialized with database integration and HLS support")
        logger.info(f"Database type: {db_type}")

    def process_audio(self, track_id):
        s3_key = self.raw_prefix + track_id

        """Process a single audio file with database tracking and HLS generation"""
        logger.info(f"="*80)
        logger.info(f"Starting processing of file: {s3_key}")

        if not track_id:
            logger.error(f"Could not extract track ID from {s3_key}")
            return False

        # Update database: processing started
        self.db_manager.update_upload_status(track_id, 'processing')

        try:
            filename = os.path.basename(s3_key)
            file_ext = os.path.splitext(filename)[1].lower()
            raw_local_path = f"/tmp/raw_{filename}"
            file_folder = f"{os.path.splitext(filename)[0]}"

            logger.info(f"Processing track ID {track_id}: {filename}")

            # Download raw file
            self.download_file(s3_key, raw_local_path)

            # Normalize loudness
            logger.info(f"Starting loudness normalization")
            if not self.normalize_loudness(raw_local_path):
                error_msg = f"Loudness normalization failed for {filename}"
                logger.error(error_msg)
                self.db_manager.update_upload_status(track_id, 'failed', error_message=error_msg)
                raise Exception(error_msg)

            # Convert to AAC (instead of OGG)
            aac_path = f"/tmp/{os.path.splitext(filename)[0]}{self.quality_profile['extension']}"
            logger.info(f"Converting to AAC: {aac_path}")

            if not self.convert_to_aac(raw_local_path, aac_path):
                error_msg = f"Conversion to AAC failed"
                logger.error(error_msg)
                self.db_manager.update_upload_status(track_id, 'failed', error_message=error_msg)
                raise Exception(error_msg)

            # Upload processed AAC file
            processed_s3_key = f"{self.processed_prefix}{file_folder}/{os.path.splitext(filename)[0]}{self.quality_profile['extension']}"
            self.upload_file(aac_path, processed_s3_key)

            # Generate HLS segments and playlists
            logger.info(f"Starting HLS generation")
            hls_output_dir = f"/tmp/hls_{os.path.splitext(filename)[0]}"


            if not self.generate_hls_streams(aac_path, hls_output_dir, file_folder):
                error_msg = f"HLS generation failed"
                logger.error(error_msg)
                self.db_manager.update_upload_status(file_folder, 'failed', error_message=error_msg)
                raise Exception(error_msg)

            # Upload HLS files to S3
            hls_s3_prefix = f"{self.processed_prefix}{file_folder}"
            if not self.upload_hls_to_s3(hls_output_dir, hls_s3_prefix):
                error_msg = f"HLS upload to S3 failed"
                logger.error(error_msg)
                self.db_manager.update_upload_status(file_folder, 'failed', error_message=error_msg)
                raise Exception(error_msg)

            # Generate metadata
            self.generate_track_metadata(track_id, filename, aac_path, hls_s3_prefix)

            # Update database: processing completed
            self.db_manager.update_upload_status(track_id, 'completed')

            # Clean up temporary files
            logger.info(f"Cleaning up temporary files")
            self.cleanup_temp_files([raw_local_path, aac_path, hls_output_dir])

            logger.info(f"Successfully processed track {track_id}: {s3_key}")
            return True

        except Exception as e:
            error_msg = f"Failed to process {s3_key}: {str(e)}"
            logger.error(error_msg)
            self.db_manager.update_upload_status(track_id, 'failed', error_message=str(e))
            return False

    def generate_hls_streams(self, input_path, output_dir, track_id):
        """Generate HLS streams for multiple quality levels"""
        try:
            os.makedirs(output_dir, exist_ok=True)

            # Process each quality level
            success_count = 0
            for quality, config in self.hls_quality_configs.items():
                quality_dir = os.path.join(output_dir, quality)
                if self.process_hls_quality(input_path, quality_dir, quality, config):
                    success_count += 1
                    logger.info(f"Successfully generated HLS for quality: {quality}")
                else:
                    logger.error(f"Failed to generate HLS for quality: {quality}")

            if success_count == 0:
                logger.error("Failed to generate HLS for any quality level")
                return False

            # Generate master playlist
            master_playlist_path = os.path.join(output_dir, 'playlist.m3u8')
            print(f"--{master_playlist_path}")
            master_content = self.generate_master_playlist(track_id)

            with open(master_playlist_path, 'w') as f:
                f.write(master_content)

            logger.info(f"Generated master playlist: {master_playlist_path}")
            return True

        except Exception as e:
            logger.error(f"Error generating HLS streams: {str(e)}")
            return False

    def process_hls_quality(self, input_path, output_dir, quality, config):
        """Process audio file for a specific HLS quality level"""
        try:
            os.makedirs(output_dir, exist_ok=True)
            playlist_path = os.path.join(output_dir, 'playlist.m3u8')

            # FFmpeg command for HLS generation
            cmd = [
                'ffmpeg',
                '-i', input_path,
                '-c:a', config['codec'],
                *config['codec_options'],
                '-b:a', config['bitrate'],
                '-ar', config['sample_rate'],
                '-ac', config['channels'],
                '-f', 'hls',
                '-hls_time', str(self.segment_duration),
                '-hls_list_size', '0',
                '-hls_segment_filename', os.path.join(output_dir, 'segment_%03d.ts'),
                '-hls_playlist_type', 'vod',
                '-hls_flags', 'independent_segments',
                '-y',
                playlist_path
            ]

            # Run FFmpeg
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                check=True
            )

            # Verify playlist was created
            if not os.path.exists(playlist_path):
                raise RuntimeError("Playlist file was not created")

            return True

        except subprocess.CalledProcessError as e:
            logger.error(f"FFmpeg failed for {quality} quality: {e.stderr}")
            return False
        except Exception as e:
            logger.error(f"Error processing {quality} quality: {str(e)}")
            return False

    def generate_master_playlist(self, track_id):
        """Generate master playlist content for adaptive streaming"""
        base_path = f"/{self.processed_prefix}{track_id}"

        playlist_content = "#EXTM3U\n#EXT-X-VERSION:3\n\n"

        # Bandwidth map (bitrate + overhead approx)-
        bandwidth_map = {
            'low': 64000,
            'med': 96000,
            'high': 160000
        }

        for quality in ['low', 'med', 'high']:
            config = self.hls_quality_configs[quality]
            bandwidth = bandwidth_map[quality]

            # AAC LC codec string for HLS
            codec_str = "mp4a.40.2"

            playlist_content += f'#EXT-X-STREAM-INF:BANDWIDTH={bandwidth},CODECS="{codec_str}"\n'
            playlist_content += f'{base_path}/{quality}/playlist.m3u8\n\n'

        return playlist_content

    def upload_hls_to_s3(self, local_dir, s3_prefix):
        """Upload HLS directory and all its contents to S3"""
        try:
            for root, dirs, files in os.walk(local_dir):
                for file in files:
                    local_file_path = os.path.join(root, file)
                    # Calculate relative path from local_dir
                    relative_path = os.path.relpath(local_file_path, local_dir)
                    s3_key = f"{s3_prefix}/{relative_path}".replace('\\', '/')

                    # Upload file with appropriate content type
                    content_type = self.get_content_type(file)
                    self.s3.upload_file(
                        local_file_path,
                        self.bucket,
                        s3_key,
                        ExtraArgs={'ContentType': content_type}
                    )
                    logger.info(f"Uploaded HLS file: {s3_key}")

            return True

        except Exception as e:
            logger.error(f"Failed to upload HLS directory {local_dir}: {str(e)}")
            return False

    def get_content_type(self, filename):
        """Get appropriate content type for file"""
        extension = os.path.splitext(filename)[1].lower()
        content_types = {
            '.m3u8': 'application/vnd.apple.mpegurl',
            '.ts': 'video/mp2t',
            '.json': 'application/json',
            '.m4a': 'audio/mp4',
            '.aac': 'audio/aac'
        }
        return content_types.get(extension, 'application/octet-stream')

    def generate_track_metadata(self, track_id, filename, aac_path, hls_s3_prefix):
        """Generate and upload track metadata"""
        try:
            # Get audio information
            audio_info = self.get_audio_info(aac_path)

            metadata = {
                'id': track_id,
                'title': audio_info.get('title', os.path.splitext(filename)[0]),
                'artist': audio_info.get('artist', 'Unknown Artist'),
                'album': audio_info.get('album', ''),
                'duration': int(audio_info.get('duration', 0)),
                'processed_at': datetime.now().isoformat(),
                'qualities': list(self.hls_quality_configs.keys()),
                'hls_master_playlist': f"{hls_s3_prefix}/playlist.m3u8",
                'original_info': {
                    'bitrate': audio_info.get('bitrate', 0),
                    'sample_rate': audio_info.get('sample_rate', 44100),
                    'channels': audio_info.get('channels', 2),
                    'codec': audio_info.get('codec', 'unknown')
                }
            }

            # Upload metadata to S3
            metadata_key = f"{hls_s3_prefix}/metadata.json"
            self.s3.put_object(
                Bucket=self.bucket,
                Key=metadata_key,
                Body=json.dumps(metadata, indent=2, ensure_ascii=False),
                ContentType='application/json'
            )

            logger.info(f"Generated metadata for track: {track_id}")
            return True

        except Exception as e:
            logger.error(f"Failed to generate metadata: {str(e)}")
            return False

    def get_audio_info(self, input_path):
        """Get audio file information using ffprobe"""
        try:
            cmd = [
                'ffprobe', '-v', 'quiet', '-print_format', 'json',
                '-show_format', '-show_streams', input_path
            ]

            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            data = json.loads(result.stdout)

            # Extract relevant information
            format_info = data.get('format', {})
            audio_stream = next(
                (s for s in data.get('streams', []) if s.get('codec_type') == 'audio'),
                {}
            )

            return {
                'duration': float(format_info.get('duration', 0)),
                'bitrate': int(format_info.get('bit_rate', 0)),
                'title': format_info.get('tags', {}).get('title', ''),
                'artist': format_info.get('tags', {}).get('artist', 'Unknown Artist'),
                'album': format_info.get('tags', {}).get('album', ''),
                'sample_rate': int(audio_stream.get('sample_rate', 44100)),
                'channels': int(audio_stream.get('channels', 2)),
                'codec': audio_stream.get('codec_name', 'unknown')
            }

        except Exception as e:
            logger.error(f"Failed to get audio info for {input_path}: {str(e)}")
            return {
                'duration': 180.0,
                'bitrate': 128000,
                'title': '',
                'artist': 'Unknown Artist',
                'album': '',
                'sample_rate': 44100,
                'channels': 2,
                'codec': 'unknown'
            }

    def cleanup_temp_files(self, file_paths):
        """Clean up temporary files and directories"""
        for path in file_paths:
            try:
                if os.path.isfile(path):
                    os.remove(path)
                    logger.info(f"Removed temporary file: {path}")
                elif os.path.isdir(path):
                    shutil.rmtree(path)
                    logger.info(f"Removed temporary directory: {path}")
            except Exception as e:
                logger.warning(f"Failed to remove {path}: {str(e)}")

    def normalize_loudness(self, audio_path):
        """Format-aware loudness normalization (unchanged)"""
        logger.info(f"Starting loudness normalization for {audio_path}")

        original_ext = os.path.splitext(audio_path)[1].lower()
        normalized_path = f"{audio_path}_normalized{original_ext}"

        try:
            # Choose appropriate codec based on original format
            if original_ext == '.ogg':
                codec = 'libvorbis'
                format_name = 'ogg'
            elif original_ext == '.mp3':
                codec = 'libmp3lame'
                format_name = 'mp3'
            elif original_ext in ['.wav', '.wave']:
                codec = 'pcm_s16le'
                format_name = 'wav'
            else:
                # Default to high-quality intermediate format
                codec = 'pcm_s24le'
                format_name = 'wav'
                normalized_path = f"{audio_path}_normalized.wav"

            (
                ffmpeg
                .input(audio_path)
                .filter('loudnorm', I=self.target_loudness, LRA=11, TP=-1.5)
                .output(normalized_path, acodec=codec, f=format_name)
                .overwrite_output()
                .run(quiet=True)
            )

            # Replace original with normalized version
            os.replace(normalized_path, audio_path)
            logger.info(f"Successfully normalized loudness to {self.target_loudness} LUFS")
            return True

        except ffmpeg.Error as e:
            logger.error(f"Loudness normalization failed: {e.stderr.decode('utf8')}")
            if os.path.exists(normalized_path):
                os.remove(normalized_path)
            return False

    def convert_to_aac(self, input_path, output_path):
        """Convert audio to AAC with optimized settings"""
        profile = self.quality_profile

        logger.info(f"Converting {input_path} to AAC at {profile['bitrate']}, {profile['sample_rate']} Hz")

        try:
            (
                ffmpeg
                .input(input_path)
                .output(
                    output_path,
                    acodec='aac',
                    audio_bitrate=profile['bitrate'],
                    ar=profile['sample_rate'],
                    ac=2,
                    map_metadata=0,
                    f='ipod'
                )
                .overwrite_output()
                .run(quiet=True)
            )
            output_size = os.path.getsize(output_path) / (1024 * 1024)
            logger.info(f"Successfully converted to AAC at {profile['bitrate']}, output size: {output_size:.2f} MB")
            return True
        except ffmpeg.Error as e:
            logger.error(f"FFmpeg error: {e.stderr.decode('utf8')}")
            return False

    def download_file(self, s3_key, local_path):
        """Download file from S3 to local storage"""
        logger.info(f"Downloading file {s3_key} to {local_path}")
        self.s3.download_file(self.bucket, s3_key, local_path)
        file_size = os.path.getsize(local_path) / (1024 * 1024)
        logger.info(f"Downloaded {s3_key} to {local_path} ({file_size:.2f} MB)")

    def upload_file(self, local_path, s3_key):
        """Upload file to S3"""
        file_size = os.path.getsize(local_path) / (1024 * 1024)
        logger.info(f"Uploading {local_path} to {s3_key} ({file_size:.2f} MB)")
        self.s3.upload_file(local_path, self.bucket, s3_key)
        logger.info(f"Successfully uploaded {local_path} to {s3_key}")

    def get_pending_tracks(self):
        """Get tracks that need processing from database"""
        try:
            with self.db_manager.get_connection() as conn:
                if self.db_manager.db_type == 'mysql':
                    cursor = conn.cursor(dictionary=True)
                else:
                    cursor = conn.cursor(cursor_factory=RealDictCursor)

                # Get tracks that haven't been processed yet
                query = """
                SELECT upload_id,file_name FROM Uploads WHERE upload_type='track' and processing_status='0' ORDER BY uploaded_at desc
                """
                cursor.execute(query)
                results = cursor.fetchall()
                print(results)

                return [dict(row) for row in results]

        except Exception as e:
            logger.error(f"Failed to get pending tracks: {str(e)}")
            return []

    def process_pending_tracks(self):
        """Process all tracks that are pending in the database"""
        pending_tracks = self.get_pending_tracks()
        logger.info(f"Found {len(pending_tracks)} pending tracks to process")

        successful_count = 0
        failed_count = 0

        for track in pending_tracks:
            upload_id = track['upload_id']
            file_name = track['file_name']

            logger.info(f"Processing upload {upload_id}: {track['file_name']}")

            if self.process_audio(file_name):
                successful_count += 1
            else:
                failed_count += 1

        logger.info(f"Processing complete. Success: {successful_count}, Failed: {failed_count}")
        return successful_count, failed_count

In [ ]:
logger.info("Starting audio processing job")
start_time = datetime.now()

# Initialize and run processor
processor = AudioProcessor(
      s3_bucket=S3_BUCKET,
      db_config=db_config,
      db_type='mysql'  # or 'postgresql'
  )

processor.process_pending_tracks()

end_time = datetime.now()
execution_time = (end_time - start_time).total_seconds()
logger.info(f"Audio processing job completed in {execution_time:.2f} seconds")

2025-06-14 08:21:46 - INFO - Starting audio processing job
2025-06-14 08:21:46 - INFO - AudioProcessor initialized with database integration and HLS support
2025-06-14 08:21:46 - INFO - Database type: mysql
2025-06-14 08:21:47 - INFO - package: mysql.connector.plugins
2025-06-14 08:21:47 - INFO - plugin_name: mysql_native_password
2025-06-14 08:21:47 - INFO - AUTHENTICATION_PLUGIN_CLASS: MySQLNativePasswordAuthPlugin
2025-06-14 08:21:48 - INFO - Found 884 pending tracks to process
2025-06-14 08:21:48 - INFO - Processing upload 1296: 20250412_926a193737f54825a59845aac73ff14c.mp3
2025-06-14 08:21:48 - INFO - ================================================================================
2025-06-14 08:21:48 - INFO - Starting processing of file: tracks/20250412_926a193737f54825a59845aac73ff14c.mp3


[{'upload_id': 1296, 'file_name': '20250412_926a193737f54825a59845aac73ff14c.mp3'}, {'upload_id': 1809, 'file_name': '20250412_2868911d351a464ebc67ac03d5a85672.mp3'}, {'upload_id': 1336, 'file_name': '20250412_926a193737f54825a59845aac73ff14c.mp3'}, {'upload_id': 1861, 'file_name': '20250412_2868911d351a464ebc67ac03d5a85672.mp3'}, {'upload_id': 1869, 'file_name': '20250412_f2d5f5bd6ebd4b02819945694cc94fe0.mp3'}, {'upload_id': 894, 'file_name': '20250412_1f168edd67c343a0a42ce1352b44d4a5.mp3'}, {'upload_id': 1150, 'file_name': '20250412_de090a1918da4d05b396539538760cbe.mp3'}, {'upload_id': 1406, 'file_name': '20250412_cc7ad7ed9d694e2cb411e5b0324facfa.mp3'}, {'upload_id': 1662, 'file_name': '20250412_e0caaad33cd84001be95a9ec319208b5.mp3'}, {'upload_id': 127, 'file_name': '20250412_3016c6397c5d48a1a68a7bc519cddf95.mp3'}, {'upload_id': 383, 'file_name': '20250412_18117458940640dcbd1ed9d3a5ecd2b8.mp3'}, {'upload_id': 639, 'file_name': '20250412_4b26c26077af43fc9e43724aedf29380.mp3'}, {'uploa

2025-06-14 08:21:49 - INFO - Found upload_id 1296 for track_id 20250412_926a193737f54825a59845aac73ff14c.mp3
2025-06-14 08:21:49 - ERROR - Database update failed for track 20250412_926a193737f54825a59845aac73ff14c.mp3: Unread result found
2025-06-14 08:21:49 - INFO - Processing track ID 20250412_926a193737f54825a59845aac73ff14c.mp3: 20250412_926a193737f54825a59845aac73ff14c.mp3
2025-06-14 08:21:49 - INFO - Downloading file tracks/20250412_926a193737f54825a59845aac73ff14c.mp3 to /tmp/raw_20250412_926a193737f54825a59845aac73ff14c.mp3
2025-06-14 08:21:49 - INFO - Downloaded tracks/20250412_926a193737f54825a59845aac73ff14c.mp3 to /tmp/raw_20250412_926a193737f54825a59845aac73ff14c.mp3 (4.97 MB)
2025-06-14 08:21:49 - INFO - Starting loudness normalization
2025-06-14 08:21:49 - INFO - Starting loudness normalization for /tmp/raw_20250412_926a193737f54825a59845aac73ff14c.mp3
2025-06-14 08:22:10 - INFO - Successfully normalized loudness to -14 LUFS
2025-06-14 08:22:10 - INFO - Converting to AAC

--/tmp/hls_20250412_926a193737f54825a59845aac73ff14c/playlist.m3u8


2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/playlist.m3u8
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_014.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_001.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_016.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_002.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_008.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac73ff14c/low/segment_007.ts
2025-06-14 08:22:22 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_926a193737f54825a59845aac

...

2025-06-14 09:18:35 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/playlist.m3u8
2025-06-14 09:18:35 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_014.ts
2025-06-14 09:18:35 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_001.ts
2025-06-14 09:18:35 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_002.ts
2025-06-14 09:18:35 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_008.ts
2025-06-14 09:18:36 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_007.ts
2025-06-14 09:18:36 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a928e12167/low/segment_009.ts
2025-06-14 09:18:36 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_4f4fbe2bc3c94f69b1bdc1a92

--/tmp/hls_20250412_1b244453b53749eba05280b84d15ff27/playlist.m3u8


2025-06-14 09:19:29 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/playlist.m3u8
2025-06-14 09:19:29 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_014.ts
2025-06-14 09:19:29 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_001.ts
2025-06-14 09:19:29 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_016.ts
2025-06-14 09:19:30 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_024.ts
2025-06-14 09:19:30 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_002.ts
2025-06-14 09:19:30 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84d15ff27/low/segment_018.ts
2025-06-14 09:19:30 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_1b244453b53749eba05280b84

--/tmp/hls_20250412_9d5e016355a846468259a24da8683680/playlist.m3u8


2025-06-14 09:20:09 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/playlist.m3u8
2025-06-14 09:20:09 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_014.ts
2025-06-14 09:20:09 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_001.ts
2025-06-14 09:20:09 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_016.ts
2025-06-14 09:20:10 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_002.ts
2025-06-14 09:20:10 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_008.ts
2025-06-14 09:20:10 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da8683680/low/segment_007.ts
2025-06-14 09:20:10 - INFO - Uploaded HLS file: mwonya_audio_tracks/20250412_9d5e016355a846468259a24da